In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/g-research-crypto-forecasting/example_sample_submission.csv
/kaggle/input/g-research-crypto-forecasting/asset_details.csv
/kaggle/input/g-research-crypto-forecasting/example_test.csv
/kaggle/input/g-research-crypto-forecasting/train.csv
/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/__init__.py


In [4]:
pip install pystan

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install prophet

     |████████████████████████████████| 65 kB 283 kB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 49 kB 738 kB/s            
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=9435399 sha256=dbb8db2a42becb2854860300a1a545d511398f777dd92d5a89f3c20c55b66431
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install fbprophet

  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.68
    Uninstalling cmdstanpy-0.9.68:
      Successfully uninstalled cmdstanpy-0.9.68
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.0.1 requires cmdstanpy==0.9.68, but you have cmdstanpy 0.9.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pydlm

     |████████████████████████████████| 46 kB 165 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for pydlm: filename=pydlm-0.1.1.11-py2.py3-none-any.whl size=58995 sha256=26dbd966eaf5c652f0b237883748ec7e001ba59f0db390231f89d4874e04e24b
  Stored in directory: /root/.cache/pip/wheels/1d/fb/0c/6611fce0b9f3611ac6498dbddb7a9b5e67c09684acf6db285d
Successfully built pydlm
Note: you may need to restart the kernel to use updated packages.


In [8]:
from datetime import datetime
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from fbprophet import Prophet

In [9]:
crypto_df = pd.read_csv(r"../input/g-research-crypto-forecasting/train.csv")
btc = crypto_df[crypto_df["Asset_ID"]==1].set_index("timestamp") # Asset_ID = 1 for Bitcoin
eth = crypto_df[crypto_df["Asset_ID"]==6].set_index("timestamp")
btc = btc.reindex(range(btc.index[0],btc.index[-1]+60,60),method='pad')
eth = eth.reindex(range(eth.index[0],eth.index[-1]+60,60),method='pad')

In [10]:
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)


upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

In [11]:
X_btc = pd.concat([log_return(btc.VWAP,periods=5), log_return(btc.VWAP,periods=1).abs(),
               upper_shadow(btc), lower_shadow(btc)], axis=1)
y_btc = btc.Target

X_eth = pd.concat([log_return(eth.VWAP,periods=5), log_return(eth.VWAP,periods=1).abs(),
               upper_shadow(eth), lower_shadow(eth)], axis=1)
y_eth = eth.Target
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))
train_window = [totimestamp("01/04/2021"), totimestamp("31/05/2021")]
test_window = [totimestamp("01/06/2021"), totimestamp("30/06/2021")]

In [13]:
X_btc_train = X_btc.loc[train_window[0]:train_window[1]].fillna(0)  # filling NaN's with zeros
y_btc_train = y_btc.loc[train_window[0]:train_window[1]].fillna(0)

X_btc_test = X_btc.loc[test_window[0]:test_window[1]].fillna(0)
y_btc_test = y_btc.loc[test_window[0]:test_window[1]].fillna(0)

In [84]:
KK = btc.Close
volume = btc.Volume
kk_new = KK.loc[train_window[0]:train_window[1]]
kk_new = kk_new[:2000]
kk_test = KK.loc[train_window[0]:train_window[1]]
kk_test = kk_test[2000:2400]
volume_new = volume.loc[train_window[0]:train_window[1]]
volume_new = volume_new[:2000]
volume_test = volume.loc[train_window[0]:train_window[1]]
volume_test = volume_test[2000:2400]
# simple preprocessing of the data
X = kk_new.values
y = kk_test.values
v_new = volume_new.values
v_test = volume_test.values
history = [x for x in X]
historical_volume = [i for i in v_new]
predictions = []
real =[]

In [85]:
df = pd.DataFrame(columns = ['ds', 'y'])
kk_new = kk_new.reset_index()

In [86]:
for i in range(len(kk_new.timestamp)):
    kk_new.at[i,'timestamp'] = datetime.fromtimestamp(kk_new.loc[i,'timestamp'])

In [87]:
kk_new = kk_new.rename(columns={"timestamp": "ds", "Close": "y"})

In [88]:
history = kk_new

In [54]:
history

,ds,y
0,2021-04-01 00:00:00,58771.605714
1,2021-04-01 00:01:00,58838.188571
2,2021-04-01 00:02:00,58958.025714
3,2021-04-01 00:03:00,59070.442857
4,2021-04-01 00:04:00,59075.228750
...,...,...
1995,2021-04-02 09:15:00,59386.908571
1996,2021-04-02 09:16:00,59444.612857
1997,2021-04-02 09:17:00,59397.644286
1998,2021-04-02 09:18:00,59405.464286


In [89]:
kk_test = kk_test.reset_index()

In [90]:
for i in range(len(kk_test.timestamp)):
    kk_test.at[i,'timestamp'] = datetime.fromtimestamp(kk_test.loc[i,'timestamp'])

In [57]:
kk_test

,timestamp,Close
0,2021-04-02 09:20:00,59275.361250
1,2021-04-02 09:21:00,59355.270000
2,2021-04-02 09:22:00,59361.735714
3,2021-04-02 09:23:00,59376.371429
4,2021-04-02 09:24:00,59442.087143
...,...,...
395,2021-04-02 15:55:00,59448.578571
396,2021-04-02 15:56:00,59455.332857
397,2021-04-02 15:57:00,59486.775714
398,2021-04-02 15:58:00,59480.929697


In [59]:
testtimeline = kk_test[:3]

In [62]:
model = Prophet()
fitted = model.fit(history)

Initial log joint probability = -2.04912
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       10907.6    0.00208502       12634.2           1           1      148   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       11147.7    0.00153495         29860           1           1      261   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       11278.2    0.00495029       7630.59           1           1      373   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       11412.6    0.00483964       4947.58           1           1      496   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       11463.5    0.00573107       26068.7           1           1      620   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [75]:
testtimeline = kk_test[:3]

In [76]:
testtimeline = kk_test.drop(columns=['Close'])
testtimeline = testtimeline.rename(columns={"timestamp": "ds"})

In [77]:
forecast = model.predict(testtimeline)

In [73]:
testtimeline

,timestamp,Close
3,2021-04-02 09:23:00,59376.371429
4,2021-04-02 09:24:00,59442.087143
5,2021-04-02 09:25:00,59436.359174


In [78]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2021-04-02 09:23:00,59443.323282,59313.366089,59554.732432,59443.323282,59443.323282,0.0,0.0,0.0,0.0,0.0,0.0,59443.323282
1,2021-04-02 09:24:00,59442.574036,59321.051696,59565.752894,59442.574036,59442.574036,0.0,0.0,0.0,0.0,0.0,0.0,59442.574036
2,2021-04-02 09:25:00,59441.824790,59318.165203,59570.749696,59441.824790,59441.824790,0.0,0.0,0.0,0.0,0.0,0.0,59441.824790
3,2021-04-02 09:26:00,59441.075544,59326.792043,59567.714008,59441.075544,59441.075544,0.0,0.0,0.0,0.0,0.0,0.0,59441.075544
4,2021-04-02 09:27:00,59440.326298,59322.523529,59564.898447,59440.326298,59440.326298,0.0,0.0,0.0,0.0,0.0,0.0,59440.326298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,2021-04-02 15:55:00,59149.618823,54767.371876,63695.203776,54759.077018,63724.922057,0.0,0.0,0.0,0.0,0.0,0.0,59149.618823
393,2021-04-02 15:56:00,59148.869577,54709.116167,63793.125040,54712.108955,63735.544653,0.0,0.0,0.0,0.0,0.0,0.0,59148.869577
394,2021-04-02 15:57:00,59148.120331,54678.447413,63708.218718,54681.952447,63746.167249,0.0,0.0,0.0,0.0,0.0,0.0,59148.120331
395,2021-04-02 15:58:00,59147.371085,54672.089913,63782.186916,54683.495247,63756.557926,0.0,0.0,0.0,0.0,0.0,0.0,59147.371085


In [69]:
yhat = forecast.loc[2,'yhat']
yhat

59441.82478980063

In [81]:
kk_test.reset_index(drop=True)

,timestamp,Close
0,2021-04-02 09:23:00,59376.371429
1,2021-04-02 09:24:00,59442.087143
2,2021-04-02 09:25:00,59436.359174
3,2021-04-02 09:26:00,59419.675714
4,2021-04-02 09:27:00,59451.841429
...,...,...
392,2021-04-02 15:55:00,59448.578571
393,2021-04-02 15:56:00,59455.332857
394,2021-04-02 15:57:00,59486.775714
395,2021-04-02 15:58:00,59480.929697


In [83]:
predictions.append(yhat)
obs = kk_test.iloc[0,1]
obs_time = kk_test.iloc[0,0]
df2 = {"ds":obs_time,
                             "y":obs}
history.append(df2,ignore_index=True)
real_price = kk_test.iloc[2,1]
real.append(real_price)
kk_test=kk_test.iloc[1:]
kk_test.reset_index(drop=True)
print('predicted=%f, expected=%f' % (yhat, y[t+2]))

predicted=59441.824790, expected=59436.359174


In [91]:
for t in range(len(kk_test)):
    try:
        testtimeline = kk_test[:3]
        kk_test.reset_index(drop=True)
        testtimeline = kk_test.drop(columns=['Close'])
        testtimeline = testtimeline.rename(columns={"timestamp": "ds"})
        model = Prophet()
        fitted = model.fit(history)
        forecast = model.predict(testtimeline)
        yhat = forecast.loc[2,'yhat']
        predictions.append(yhat)
        obs = kk_test.iloc[0,1]
        obs_time = kk_test.iloc[0,0]
        df2 = {"ds":obs_time,
                             "y":obs}
        history.append(df2,ignore_index=True)
        real_price = kk_test.iloc[2,1]
        real.append(real_price)
        kk_test=kk_test.iloc[1:]
        kk_test.reset_index(drop=True)
        print('predicted=%f, expected=%f' % (yhat, y[t+2]))
    except:
        break
predictions = predictions[:len(real)]
print('Test score for LR baseline: BTC', f"{np.corrcoef(predictions, real)[0,1]:.2f}")

Initial log joint probability = -2.04912
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       10907.6    0.00208502       12634.2           1           1      148   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       11147.7    0.00153495         29860           1           1      261   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       11278.2    0.00495029       7630.59           1           1      373   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       11412.6    0.00483964       4947.58           1           1      496   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       11463.5    0.00573107       26068.7           1           1      620   
    Iter      log prob        ||dx||      ||grad||       alpha  